In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests

In [2]:
df=pd.read_csv('Compilation webscrapping cosmetique.csv')

In [3]:
df.head()

,company,client,nb_avis_client,date_commande,notes,titre_commentaire,commentaire,date_commentaire,reponse_sav,date_reponse_sav,delai_reponse
0,Sephora,MARIANNE BAILLEUL,4,2023-10-03,1,Conseils maquillages nuls,Conseils maquillages nuls ! un mascara qui fai...,2023-10-04 13:40:17+00:00,NaN,NaN,NaN
1,Sephora,valerie Tixier,9,2022-12-06,1,Colis non encore reçu après deux…,Colis non encore reçu après deux semaines - il...,2022-12-07 23:23:02+00:00,NaN,NaN,NaN
2,Sephora,Suzy B,1,2023-05-25,5,ACCUEIL AGREABLE ET CHALEUREUX,J'ai été merveilleusement accueillie et consei...,2023-05-26 22:28:26+00:00,NaN,NaN,NaN
3,Sephora,Nam Nam,1,2020-08-20,1,Fuyez!,"Mais plus jamais, jamais , jamais!!! Sensé ouv...",2020-08-20 08:34:01+00:00,NaN,NaN,NaN
4,Sephora,Claire Desmedt,3,2022-05-30,1,Toujours pas de remboursement après un…,Toujours pas de remboursement après un mois et...,2022-05-30 13:56:52+00:00,NaN,NaN,NaN


In [4]:
df['company'].value_counts()

company
My_origines    2680
Marionnaud     1980
Nocibé         1180
Sephora         353
Name: count, dtype: int64

In [5]:
# on élimine d'abord le nom des marques de commentaires car ce dernier influe les prédictions de notes

liste_marques = ["Nocibé", "Nocibe", "nocibe", "Sephora","Séphora", "sephora", "Marionnaud", "marionnaud","my origns",
                 "My Origns", "My Origines"]
for marque in liste_marques:
    df['commentaire'] = df['commentaire'].str.replace(marque, '', case=False)
    
# le séphora en ligne 5 a bien été supprimé de 'commentaire' mais persiste dans le 'titre du commentaire' que nous 
# n'utilisons pas

In [6]:
"""
Pour mémoire
Le pourcentage de répartition des étoiles est de :  notes
5    58.088235
1    26.720901
4     8.307259
2     3.817272
3     3.066333
"""

# nous allons sélectionner ~2000 données au total (dans un premier temps ) soit
# 581 notes 5; 267 notes 1; 83 notes 4; 38 notes 2; 31 notes 3

notes_5 = df[df['notes'] == 5].sample(n=581, random_state=42)
notes_1 = df[df['notes'] == 1].sample(n=267, random_state=42)
notes_4 = df[df['notes'] == 4].sample(n=83, random_state=42)
notes_2 = df[df['notes'] == 2].sample(n=38, random_state=42)
notes_3 = df[df['notes'] == 3].sample(n=31, random_state=42)

# on appelle df_reduit notre dataframe avec moins de données
df_reduit = pd.concat([notes_5, notes_1, notes_4, notes_2, notes_3])

In [7]:
# Import des packages nécessaires pour les différents algorithmes

from sklearn import model_selection
from sklearn import ensemble

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn import svm, neighbors
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, classification_report

In [8]:
# On ne garde de notre dataframe que les commentaires (variable explicative) et les notes (variable à prédire)
X, y = df_reduit.commentaire, df_reduit.notes

# Séparation des données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Définition de l'objet CountVectorizer() et du GradientBoostingClassifier()
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [9]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler,  ClusterCentroids

smo = SMOTE()
X_sm, y_sm = smo.fit_resample(X_train, y_train)

In [10]:
import time

In [11]:
svc = SVC()

params_svc = {"kernel":["linear","sigmoid","rbf"], "C" : [0.1,1,10]}

grid_svc=GridSearchCV(estimator = svc, param_grid = params_svc)

time_svc_debut = time.time()
grid_svc.fit(X_sm, y_sm)
time_svc_fin = time.time()

print("Les meilleurs hyperparamètres sont : ", grid_svc.best_params_)

y_pred_svc = grid_svc.predict(X_test)

print(pd.crosstab(y_test, y_pred_svc))

print(grid_svc.score(X_test, y_test))

print( classification_report(y_test, y_pred_svc) )

print('Le temps mis par le SVC est de : ', time_svc_fin - time_svc_debut, ' secondes.')

Les meilleurs hyperparamètres sont :  {'C': 10, 'kernel': 'rbf'}
col_0   1  2  3   4   5
notes                  
1      38  6  1   0   6
2       3  1  0   1   1
3       3  0  0   0   5
4       2  0  1   6   9
5       8  3  5  11  90
0.675
              precision    recall  f1-score   support

           1       0.70      0.75      0.72        51
           2       0.10      0.17      0.12         6
           3       0.00      0.00      0.00         8
           4       0.33      0.33      0.33        18
           5       0.81      0.77      0.79       117

    accuracy                           0.68       200
   macro avg       0.39      0.40      0.39       200
weighted avg       0.69      0.68      0.68       200

Le temps mis par le SVC est de :  118.41145825386047  secondes.


In [12]:
knn = neighbors.KNeighborsClassifier()

params_knn={"n_neighbors" : range(2,41)}

grid_knn=GridSearchCV(estimator = knn, param_grid = params_knn, cv=5)

time_knn_debut = time.time()
grid_knn.fit(X_sm, y_sm)
time_knn_fin = time.time()

print( grid_knn.best_params_)

y_pred_knn = grid_knn.predict(X_test)

print(pd.crosstab(y_test, y_pred_knn))

print(grid_knn.score(X_test, y_test))

print( classification_report(y_test, y_pred_knn) )

print('Le temps mis par le KNN est de : ', time_knn_fin - time_knn_debut, ' secondes.')

{'n_neighbors': 2}
col_0   1   2   3   4   5
notes                    
1      31  14   4   2   0
2       2   3   1   0   0
3       2   1   1   2   2
4       5   3   2   6   2
5      23  15  16  34  29
0.35
              precision    recall  f1-score   support

           1       0.49      0.61      0.54        51
           2       0.08      0.50      0.14         6
           3       0.04      0.12      0.06         8
           4       0.14      0.33      0.19        18
           5       0.88      0.25      0.39       117

    accuracy                           0.35       200
   macro avg       0.33      0.36      0.27       200
weighted avg       0.66      0.35      0.39       200

Le temps mis par le KNN est de :  28.07699203491211  secondes.


In [13]:
rf = RandomForestClassifier()

params_rf = {'n_estimators': [100, 200, 300], 'n_jobs': [-1, 1, 2]}

grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=5)

time_rf_debut = time.time()
grid_rf.fit(X_sm, y_sm)
time_rf_fin = time.time()

print( grid_rf.best_params_)

y_pred_rf = grid_rf.predict(X_test)

print(pd.crosstab(y_test, y_pred_rf))

print(grid_rf.score(X_test, y_test))

print( classification_report(y_test, y_pred_rf) )

print('Le temps mis par le Random Forest est de : ', time_rf_fin - time_rf_debut, ' secondes.')

{'n_estimators': 200, 'n_jobs': -1}
col_0   1  2  3   4   5
notes                  
1      49  1  0   0   1
2       4  1  0   1   0
3       3  0  0   2   3
4       2  2  1   4   9
5       6  2  3  12  94
0.74
              precision    recall  f1-score   support

           1       0.77      0.96      0.85        51
           2       0.17      0.17      0.17         6
           3       0.00      0.00      0.00         8
           4       0.21      0.22      0.22        18
           5       0.88      0.80      0.84       117

    accuracy                           0.74       200
   macro avg       0.40      0.43      0.41       200
weighted avg       0.73      0.74      0.73       200

Le temps mis par le Random Forest est de :  591.502447605133  secondes.


In [14]:
gb = GradientBoostingClassifier()

params_gb = {'n_estimators': [50, 100, 150],'learning_rate': [0.01, 0.1, 0.2],'max_depth': [3, 4, 5]}

grid_gb = GridSearchCV(estimator=gb, param_grid=params_gb, cv=5)

time_gb_debut = time.time()
grid_gb.fit(X_sm, y_sm)
time_gb_fin = time.time()

print( grid_gb.best_params_)

y_pred_gb = grid_gb.predict(X_test)

print(pd.crosstab(y_test, y_pred_gb))

print(grid_gb.score(X_test, y_test))

print( classification_report(y_test, y_pred_gb) )

print('Le temps mis par le Gradient Boosting est de : ', time_gb_fin - time_gb_debut, ' secondes.')

{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}
col_0   1  2  3   4   5
notes                  
1      39  4  2   1   5
2       4  1  0   0   1
3       4  0  0   2   2
4       1  0  1   5  11
5       7  0  2  15  93
0.69
              precision    recall  f1-score   support

           1       0.71      0.76      0.74        51
           2       0.20      0.17      0.18         6
           3       0.00      0.00      0.00         8
           4       0.22      0.28      0.24        18
           5       0.83      0.79      0.81       117

    accuracy                           0.69       200
   macro avg       0.39      0.40      0.39       200
weighted avg       0.69      0.69      0.69       200

Le temps mis par le Gradient Boosting est de :  2552.343559026718  secondes.


In [ ]:
# Parmis les algos SVM, KNN, Random Forest, Gradient Boosting entrainés sur notre jeu de données équilibré et réduit, la meilleure accuracy est obtenue avec le 